In [1]:
import openfl.native as fx
from openfl.federated import FederatedModel, FederatedDataSet

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fx.init('torch_cnn_mnist', log_level='METRIC', log_file='./income.log')

Creating Workspace Directories
Creating Workspace Templates


You should consider upgrading via the '/Users/zhiruzhu/Desktop/data_station/fl_test/venv/bin/python -m pip install --upgrade pip' command.



New workspace directory structure:
workspace
├── final_pytorch_model
├── spam_metric.log
├── requirements.txt
├── .workspace
├── plan
│   ├── plan.yaml
│   ├── cols.yaml
│   ├── defaults
│   └── data.yaml
├── final_income_model
├── logs
│   ├── income
│   │   └── events.out.tfevents.1652739750.MacBook-Pro-2.local.33839.5
│   └── cnn_mnist
│       ├── events.out.tfevents.1652899361.MacBook-Pro-2.local
│       ├── events.out.tfevents.1652899988.MacBook-Pro-2.local
│       ├── events.out.tfevents.1652901524.MacBook-Pro-2.local
│       └── events.out.tfevents.1651982305.MacBook-Pro-2.local.38935.0
├── cert
├── agg_to_col_one_signed_cert.zip
├── save
│   ├── torch_cnn_mnist_best.pbuf
│   ├── torch_cnn_mnist_init.pbuf
│   └── torch_cnn_mnist_last.pbuf
├── data
│   ├── MNIST
│   │   ├── processed
│   │   └── raw
│   ├── cifar-10-batches-py
│   │   ├── data_batch_1
│   │   ├── readme.html
│   │   ├── batches.meta
│   │   ├── data_batch_2
│   │   ├── data_batch_5
│   │   ├── test_batch
│   │  

In [3]:
train = pd.read_csv("/Users/zhiruzhu/Desktop/data_station/fl_test/income/train.csv")
train

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,0
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43952,52,Private,68982,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
43953,19,Private,116562,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,0
43954,30,Private,197947,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,58,United-States,0
43955,46,Private,97883,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,35,United-States,0


In [4]:
def preprocess(data):
    # remove space
    data.columns = [cols.replace(' ', '') for cols in data.columns]
    data["education"] = [cols.replace(' ', '') for cols in data["education"]]
    data["marital-status"] = [cols.replace(' ', '') for cols in data["marital-status"]]
    data["relationship"] = [cols.replace(' ', '') for cols in data["relationship"]]
    data["race"] = [cols.replace(' ', '') for cols in data["race"]]
    data["gender"] = [cols.replace(' ', '') for cols in data["gender"]]

    #missing data
    data = data.replace('?', np.nan)
    data.dropna(inplace=True, axis=0)

    #categorical value
    cat_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender',
                   'native-country']
    df_dummy = pd.get_dummies(data, columns=cat_columns)
    return df_dummy

In [5]:
train = preprocess(train)
train

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income_>50K,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,67,366425,16,99999,0,60,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,17,244602,8,0,0,15,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,31,174201,13,0,0,40,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,58,110199,4,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,25,149248,10,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43952,52,68982,13,0,0,50,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
43953,19,116562,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
43954,30,197947,10,0,0,58,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
43955,46,97883,13,0,0,35,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [6]:
from sklearn.model_selection import train_test_split

X = train.drop("income_>50K", axis=1)
y = train["income_>50K"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [7]:
X_train = X_train.to_numpy(dtype=np.float32)
X_test = X_test.to_numpy(dtype=np.float32)
y_train = y_train.to_numpy(dtype=int)
y_test = y_test.to_numpy(dtype=int)

In [8]:
def one_hot(labels, classes):
    return np.eye(classes)[labels].astype(int)

y_test = one_hot(y_test, 2)
y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])

In [9]:
y_test.dtype

dtype('int64')

In [10]:
batch_size = 32
num_classes = 2

fl_data = FederatedDataSet(X_train, y_train, X_test, y_test, batch_size=batch_size) #, num_classes=num_classes)

Inferred 2 classes from the provided labels...


In [11]:
X_train.shape[1]

104

In [12]:
input_dim = int(X_train.shape[1])
output_dim = 1

class LogisticRegression(torch.nn.Module):

     def __init__(self):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)#.float()

     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))#.float()
         return outputs


learning_rate = 0.01
optimizer = lambda x: torch.optim.SGD(x, lr=learning_rate)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    # criterion = nn.CrossEntropyLoss()
    # loss = criterion(output, target)
    # return loss
    output = torch.squeeze(output)
#     output = output.float()
    return F.binary_cross_entropy(input=output.float(),target=target.float())

In [13]:
!pwd

/Users/zhiruzhu/.local/workspace


In [14]:
import time

# log = open("./logs/income_openfl.log", "w")

def write_metric_x(node_name, task_name, metric_name, metric, round_number):
    with open("./logs/income_openfl.log", "a") as log:
#     with open("/Users/zhiruzhu/.local/workspace/logs/income_openfl.log", "a") as log:
        log.write("{}/{}/{}: {}, {}, {}\n".format(node_name, task_name, metric_name, metric, round_number, time.time()))
#     exit()
    
# from torch.utils.tensorboard import SummaryWriter

# writer = SummaryWriter('./logs/income', flush_secs=5)

# def write_metric(node_name, task_name, metric_name, metric, round_number):
#     writer.add_scalar("{}/{}/{}".format(node_name, task_name, metric_name),
#                       metric, round_number)

In [15]:
#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=LogisticRegression,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

In [16]:
num_collaborators = 8
collaborator_models = fl_model.setup(num_collaborators=num_collaborators)
collaborators = {}
for i in range(num_collaborators):
    collaborators[i] = collaborator_models[i]

In [17]:
#Original MNIST dataset
print(f'Original training data size: {len(X_train)}')
print(f'Original validation data size: {len(X_test)}\n')

for i, model in enumerate(collaborator_models):
    print(f'Collaborator {i}\'s training data size: {len(model.data_loader.X_train)}')
    print(f'Collaborator {i}\'s validation data size: {len(model.data_loader.X_valid)}\n')

Original training data size: 36654
Original validation data size: 4073

Collaborator 0's training data size: 4582
Collaborator 0's validation data size: 510

Collaborator 1's training data size: 4582
Collaborator 1's validation data size: 509

Collaborator 2's training data size: 4582
Collaborator 2's validation data size: 509

Collaborator 3's training data size: 4582
Collaborator 3's validation data size: 509

Collaborator 4's training data size: 4582
Collaborator 4's validation data size: 509

Collaborator 5's training data size: 4582
Collaborator 5's validation data size: 509

Collaborator 6's training data size: 4581
Collaborator 6's validation data size: 509

Collaborator 7's training data size: 4581
Collaborator 7's validation data size: 509



In [18]:
 #Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback": {
        "template": "src.mnist_utils.write_metric"
    },
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": false,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
  

In [19]:
# fx.setup_plan()
# fx.setup_logging(level="METRIC", log_file="./income.log")
plan = fx.update_plan(override_config={
    'aggregator.settings.rounds_to_train': 8,
    'aggregator.settings.log_metric_callback': write_metric_x,
    "aggregator.settings.write_logs": True,
})
print(plan.config)

{'task_runner': {'template': 'src.pt_cnn.PyTorchCNN', 'settings': {}}, 'compression_pipeline': {'template': 'openfl.pipelines.NoCompressionPipeline', 'settings': {}}, 'aggregator': {'template': 'openfl.component.Aggregator', 'settings': {'best_state_path': 'save/torch_cnn_mnist_best.pbuf', 'write_logs': True, 'rounds_to_train': 8, 'init_state_path': 'save/torch_cnn_mnist_init.pbuf', 'last_state_path': 'save/torch_cnn_mnist_last.pbuf', 'db_store_rounds': 2, 'log_metric_callback': <function write_metric_x at 0x148258430>}}, 'assigner': {'template': 'openfl.component.RandomGroupedAssigner', 'settings': {'task_groups': [{'name': 'train_and_validate', 'percentage': 1.0, 'tasks': ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']}]}}, 'tasks': {'settings': {}, 'train': {'function': 'train_batches', 'kwargs': {'epochs': 1, 'metrics': ['loss']}}, 'locally_tuned_model_validation': {'function': 'validate', 'kwargs': {'apply': 'local', 'metrics': ['acc']}}, 'aggregated_mod

In [22]:
# Run experiment, return trained FederatedModel

final_fl_model = fx.run_experiment(collaborators
                                   ,override_config={
    'aggregator.settings.rounds_to_train': 8,
    'aggregator.settings.log_metric_callback': write_metric_x,
    "aggregator.settings.write_logs": True,
    }
                                  )

In [21]:
final_fl_model.logger

<Logger openfl.federated.task.runner (METRIC)>

In [45]:
!cat income.log

In [28]:
#Save final model
final_fl_model.save_native('final_income_model')

In [ ]:
# log.close()

In [ ]:
# writer.close()